<a href="https://colab.research.google.com/github/ashutoshgithubs/Machine-Learning/blob/main/fakeNewsUsingBidirLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import pandas as pd

In [43]:
df = pd.read_csv("/content/fake_train.csv", engine="python",encoding="utf-8",
                 skipinitialspace=True, on_bad_lines="skip", quotechar='"')

In [44]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [45]:
df.shape

(20822, 5)

In [46]:
df.isnull().sum()

,0
id,0
title,565
author,1975
text,135
label,22


In [47]:
df = df.dropna()

In [48]:
df.shape

(18212, 5)

In [49]:
X = df.drop('label',axis=1)
y = df['label']

In [50]:
X.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [51]:
y

,label
0,1
1,0
2,1
3,1
4,1
...,...
20817,0
20818,0
20819,0
20820,1


In [52]:
messages = X.copy()

In [53]:
messages.reset_index(inplace=True) # coz i've removed some nan values so i need to reset the entire index.

### Text preprocessing

In [54]:
import nltk
import re
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [55]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [56]:
corpus=[]
for i in range(0,len(messages)):
  text_clean = text_clean = re.sub(r'[^a-zA-Z\s]', "", messages['title'][i])
  lower_text = text_clean.lower()
  tolenized_text = word_tokenize(lower_text)
  stemmed_word = []
  for word in tolenized_text:
    if not word in stopwords.words('english'):
      words = ps.stem(word)
      stemmed_word.append(words)

  review = ' '.join(stemmed_word)
  corpus.append(review)

In [57]:
corpus

['hous dem aid didnt even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'benot hamon win french socialist parti presidenti nomin new york time',
 'backchannel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soroslink indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close

In [58]:
import tensorflow as tf

In [59]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [60]:
#Vocabulary size
voc_size = 5000

#### One Hot

In [61]:
oneHot_rep = [one_hot(words,voc_size) for words in corpus]

In [62]:
print(oneHot_rep)

[[3180, 1605, 1477, 230, 4072, 1267, 3413, 4217, 2706, 2716, 178], [3683, 2847, 1752, 1401, 3125, 4680, 3764], [4337, 94, 4834, 1088], [2981, 863, 2431, 496, 2105, 3703], [4166, 3125, 3161, 1093, 4712, 2459, 3125, 4203, 3046, 4475], [4250, 2691, 4770, 3140, 3233, 2302, 4925, 864, 4018, 4702, 2366, 3796, 4449, 4373, 3764], [3643, 1678, 1276, 1861, 921, 3918, 1376, 4851, 330, 3075, 2629], [3702, 1716, 2272, 2533, 4044, 2302, 927, 330, 3075, 2629], [4223, 432, 58, 1409, 4765, 371, 2643, 2302, 1936], [1284, 3472, 4291, 2040, 3185, 14, 4796, 2236], [2181, 3704, 1048, 1664, 1097, 3181, 1373, 348, 4470, 1319, 2307], [496, 1888, 1267, 1962, 2302, 2533], [265, 631, 571, 919, 2911, 4539, 4459, 4931, 2786], [3899, 2904, 142, 4351, 1747, 1697, 3743, 330, 3075, 2629], [1430, 1885, 3544, 2666, 958, 330, 3075, 2629], [4473, 250, 4354, 2564, 1385, 3687, 2181, 2061, 724, 274], [3836, 3430, 2847], [360, 764, 2301, 1878, 2302, 4821, 4893, 3764], [4081, 2572, 1752, 2417, 4976, 3746, 2916, 2264, 2459], [11

### Embedding Representation

In [63]:
sent_length=20
embedded_docs=pad_sequences(oneHot_rep,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2706 2716  178]
 [   0    0    0 ... 3125 4680 3764]
 [   0    0    0 ...   94 4834 1088]
 ...
 [   0    0    0 ...  330 3075 2629]
 [   0    0    0 ... 2099 2713 2049]
 [   0    0    0 ... 2670 2021 4044]]


In [64]:
## Creating model
embedding_vector_features=40 # Each word is represented as a vector of size 40.
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid')) #This adds a dense (fully connected) output layer with one unit and a sigmoid activation function.
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [65]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [66]:
X_final

array([[   0,    0,    0, ..., 2706, 2716,  178],
       [   0,    0,    0, ..., 3125, 4680, 3764],
       [   0,    0,    0, ...,   94, 4834, 1088],
       ...,
       [   0,    0,    0, ...,  330, 3075, 2629],
       [   0,    0,    0, ..., 2099, 2713, 2049],
       [   0,    0,    0, ..., 2670, 2021, 4044]], dtype=int32)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [69]:
y_pred = model.predict(X_test)

188/188 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step


In [70]:
y_pred = np.where(y_pred > 0.5, 1, 0)

In [71]:
y_pred

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]])

In [75]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
confusion_matrix(y_test,y_pred)

In [ ]:
print(accuracy_score(y_test,y_pred))